Загрузить данные из таблицы test.xlsx.  
Проанализировать текст в колонке text, используя метод обработки текста.  
Сгенерировать вероятный краткий ответ из всего текста (состоящий из нескольких слов) и записать его в колонку answer.  

Пример:  
Текст1 - Удовлетворить  
Текст2 - Производство окончено  
Текст3 - Отказ, ИП приостановлено  
Текст4 - Отказ, невозможно установить местонахождение должника  

## Обработка данных

In [1]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
df = pd.read_excel('data/orig/test.xlsx')
df.head()

,Id,text,answer
0,672527f94ac0e6c927fe9209,"Обратить\t взыскание\t на\t денежные\t средства\t должника\t КОВАЛЕВ\t ВЯЧЕСЛАВ\t ГЕННАДЬЕВИЧ\t 03.09.1978\nг.р.,\tнаходящиеся\tна\tсчетах\tв\tбанках\tи\tиных\tкредитных\tорганизациях:\nЮГО-ЗАПАДНЫЙ\tБАНК\tПАО\tСБЕРБАНК,\tБИК\t046015602,\tсчет:\t40817810630061819252;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601398000003700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601398500003700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601933800003700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601933300003700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601840100003700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601840600003700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601810800003700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601810700001700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601978200003700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601810300003700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601840000001700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601978700003700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601840500001700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601810200001700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601978600001700124;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601978100001700129;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601156200003700124;\nФИЛИАЛ\tNo\t7701\tБАНКА\tВТБ\t(ПАО),\tБИК\t044525745,\tсчет:\t30601810205052519251;\nФИЛИАЛ\tNo\t2351\tБАНКА\tВТБ\t(ПАО),\tБИК\t040349758,\tсчет:\t40817840932556001971;\nФИЛИАЛ\tNo\t2351\tБАНКА\tВТБ\t(ПАО),\tБИК\t040349758,\tсчет:\t40817810832556010898;\nЮГО-ЗАПАДНЫЙ\tБАНК\tПАО\tСБЕРБАНК,\tБИК\t046015602,\tсчет:\t40817810530062328130;\nФИЛИАЛ\tNo\t7701\tБАНКА\tВТБ\t(ПАО),\tБИК\t044525745,\tсчет:\t40817810722254002807;\nФИЛИАЛ\tNo\t7701\tБАНКА\tВТБ\t(ПАО),\tБИК\t044525745,\tсчет:\t40817978222254002168;\nФИЛИАЛ\tNo\t7701\tБАНКА\tВТБ\t(ПАО),\tБИК\t044525745,\tсчет:\t40817840622254002168;\nКБ\t""Ренессанс\tКредит""\t(ООО),\tБИК\t044525135,\tсчет:\t40817810997750205910;\nПАО\tСБЕРБАНК,\tБИК\t044525225,\tсчет:\t40817810938049607994;\nПАО\tСБЕРБАНК,\tБИК\t044525225,\tсчет:\t40817810038064928749;\nПАО\tСБЕРБАНК,\tБИК\t044525225,\tсчет:\t40817810038064927313;\nПАО\tСБЕРБАНК,\tБИК\t044525225,\tсчет:\t40817810138170573768;\nПАО\tСБЕРБАНК,\tБИК\t044525225,\tсчет:\t40817810540005904391;\nПАО\tРОСБАНК,\tБИК\t044525256,\tсчет:\t40817810097440007054;\nКУРСКОЕ\tОТДЕЛЕНИЕ\tN8596\tПАО\tСБЕРБАНК,\tБИК\t043807606,\tсчет:\t42307810033000116032;\nАО\t""АЛЬФА-БАНК"",\tБИК\t044525593,\tсчет:\t40817810506370028148;\nБАНК\tГПБ\t(АО),\tБИК\t044525823,\tсчет:\t40817810964071606339;\nБАНК\tГПБ\t(АО),\tБИК\t044525823,\tсчет:\t40817810364071611665;\nФИЛИАЛ\tЦЕНТРАЛЬНЫЙ\tПАО\tБАНКА\t""ФК\tОТКРЫТИЕ"",\tБИК\t044525297,\tсчет:\t40817810978006973068;\nФИЛИАЛ\tNo\t2351\tБАНКА\tВТБ\t(ПАО),\tБИК\t040349758,\tсчет:\t40817978432556001929;\nПАО\t""ПРОМСВЯЗЬБАНК"",\tБИК\t044525555,\tсчет:\t30601156700003700129;\nнезависимо\tот\tостатка\tденежных\tсредств\tна\tсчете.",NaN
1,672526134ac0e6c927fe3400,"Обратить\t взыскание\t на\t денежные\t средства\t должника\t Арбузова\t Михаила\t Михайловича\t 03.03.1976\t г.р.,\nнаходящиеся\tна\tсчетах\tв\tбанках\tи\tиных\tкредитных\tорганизациях:\nАО\t""ТИНЬКОФФ\tБАНК"",\tБИК\t044525974,\tсчет:\t40817810600001600017;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК,\tБИК\t045354601,\tсчет:\t42307810046140126000;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК,\tБИК\t045354601,\tсчет:\t42307810646007402351;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК,\tБИК\t045354601,\tсчет:\t42307810346005600057;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК,\tБИК\t045354601,\tсчет:\t40817810146007400345;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК,\tБИК\t045354601,\tсчет:\t42307840146007400030;\nОРЕНБУРГСКОЕ\tОТДЕЛЕНИЕ\tN8623\tПАО\tСБЕРБАНК

In [3]:
# Проверяем на пустые значения в столбцах, если вдруг надо обработать их
df.isnull().sum()

Id           0
text         0
answer    1000
dtype: int64

Так как цифры встречаются очень часто и не несут смысловой нагрузки, можем обработать их, как и табуляции  

In [4]:
import re

def clean_text(text):
    # Удаляем табуляции/переносы строк, приводим к нижнему регистру и убираем знаки препинания
    text = text.replace('\t', ' ').replace('\n', ' ')
    
    # # Удаляем все цифры и даты из текста (хотя при лематизации цифры и так удаляются)
    # text = re.sub(r'\d{2}.\d{2}.\d{4}', '', text)
    # text = re.sub(r'\d+', '', text)

    # Удаляем оставшиеся латиницу, цифры, знаки препинания
    text = re.sub(r"[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+", ' ', text).lower()
    return text

In [5]:
# Применяем функции ко всем строкам
df['clean'] = df['text'].apply(clean_text)

Также нужно убрать часто встречающиеся лишние слова по типу "счёт, банк" и подобные

In [6]:
# Подсчет слов
word_counts = df['clean'].str.split().explode().value_counts()

# Вывод топ слов
word_counts.head(60)

clean
в                  6178
не                 3150
о                  2380
на                 1961
указано            1892
по                 1824
и                  1766
сведений           1661
запрос             1486
с                  1266
дата               1252
банк               1224
мвв                1074
№                  1001
должника            973
счета               770
россии              725
г                   713
нет                 697
исполнительного     661
актуальности        647
сумме               641
пао                 595
сведения            584
постановление       578
а                   564
что                 561
производства        557
счет                551
средств             545
сумма               518
связи               508
денежных            493
бик                 473
удовлетворению      468
код                 468
ип                  463
ао                  454
признает            452
валюта              450
от                  447
заявителя 

In [7]:
# Выберем те, которые кажутся лишними
# Точно надо удалить их перед проверкой орфографии как можно больше, так как слишком долго обрабатывает,
# но и искать каждое слово в каждом падеже как-то неправильно. Может снова пройтись после лемматизации, так как это может убрать ещё слова
unnecessary_words = {
    "счёт", "счет", "счета", "счете", "счетах", "банк", "банка", "банках", "дата", 
    "№", "валюта", "сумма", "сумме", "рубль", "курс", "ао", "пао", "бик", "код", "адрес",
    "мвв", "россия", "филиал", "отделение", "альфабанк", "альфа банк", "альфа", "райффайзен",
    "тинькофф", "россельхоз", "сбербанк", "промсвязьбанк", "ренессанс", "втб",
    "код", "росбанк", "кб"
}

def remove_unnecessary_words(text):
    pattern = r'\b(' + '|'.join(re.escape(word) for word in unnecessary_words) + r')\b'
    return re.sub(pattern, '', text, flags=re.IGNORECASE)

In [8]:
df['clean'] = df['clean'].apply(remove_unnecessary_words)

### Стоп-слова

In [9]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words("russian")
stopwords_ru.remove('не')

# Оставляем только "не" из всех отдельных стоп-слов для правильной лемматизации
def remove_stopwords(text):
    # Вдруг попадётся не строка или нуль
    if not isinstance(text, str) or not text.strip():
        return text
    
    tokens = text.split()
    
    filtered_tokens = [token for token in tokens 
                       if token and token not in stopwords_ru]
    
    return ' '.join(filtered_tokens)

In [10]:
df['clean'] = df['clean'].apply(remove_stopwords)

### Орфография

#### Логгирование

In [11]:
from tqdm import tqdm
import time

def print_spell_check_stats(stats, elapsed_time):
    percent_corrected = (stats['corrected_words'] / stats['total_words'] * 100) if stats['total_words'] > 0 else 0
    speed = stats['total_words'] / elapsed_time if elapsed_time > 0 else 0
    
    print(f"Всего слов обработано: {stats['total_words']:,}")
    print(f"Исправлено слов: {stats['corrected_words']:,}")
    print(f"Процент исправленных слов: {percent_corrected:.2f}%")
    print(f"Попаданий в кэш: {stats['cached_hits']:,}")
    print(f"Время выполнения: {elapsed_time:.2f} сек")
    print(f"Скорость: {speed:.0f} слов/сек")

Используем Hanspell, то есть spylls, который то же самое, но новее.  
Может стоило использовать [JamSpell](https://github.com/bakwc/JamSpell), который намного быстрее и точнее, но это только на Linux  
Так же попробуем PySpellChecker

#### Spylls

In [4]:
from spylls.hunspell import Dictionary

dictionary = Dictionary.from_zip('dictionary/russian_spellchecking_dic_3703-0.4.5.2resigned1.xpi')

word_cache = {}
stats = {
    'total_words': 0,
    'corrected_words': 0,
    'cached_hits': 0,
    'start_time': None,
}

def spell_check_spylls(text):
    # Вдруг попадётся не строка или нуль
    if not isinstance(text, str) or not text.strip():
        return text

    words = text.split()
    corrected_words = []
    
    for word in words:
        # Проверка на пустые слова или слова только из пробелов (можно убрать позже, если выявится, что слова не склеены случайно)
        if not word.strip():
            corrected_words.append(word)
            continue

        stats['total_words'] += 1
        
        # Проверяем кэш первым
        if word in word_cache:
            stats['cached_hits'] += 1
            corrected_words.append(word_cache[word])
            continue
        
        # Если нет в кэше - проверяем и кэшируем результат
        if dictionary.lookup(word):
            word_cache[word] = word
            corrected_words.append(word)
        else:
            suggestions = list(dictionary.suggest(word))
            corrected = suggestions[0] if suggestions else word

            # Если слово было исправлено
            if corrected != word:
                stats['corrected_words'] += 1
            
            word_cache[word] = corrected
            corrected_words.append(corrected)
    
    return ' '.join(corrected_words)

In [13]:
stats['start_time'] = time.time()

tqdm.pandas(desc="Spylls")
df['corrected'] = df['clean'].progress_apply(spell_check_spylls) # Долго, первые 5 строк делает 6.7 секунды, без кэша ещё в 5x медленее. А мультипроцессорность и мультипоточность не помогает

elapsed_time = time.time() - stats['start_time']
print_spell_check_stats(stats, elapsed_time)

Spylls: 100%|██████████| 1000/1000 [14:00<00:00,  1.19it/s]

Всего слов обработано: 88,992
Исправлено слов: 2,229
Процент исправленных слов: 2.50%
Попаданий в кэш: 81,947
Время выполнения: 840.56 сек
Скорость: 106 слов/сек


#### PySpellChecker

In [ ]:
from spellchecker import SpellChecker

spell_checker = SpellChecker(language='ru')
stats = {
    'total_words': 0,
    'corrected_words': 0,
    'cached_hits': 0,
    'start_time': None,
}
word_cache = {}

def spell_check_py(text):
    if not isinstance(text, str) or not text.strip():
        return text
    
    words = text.split()
    corrected_words = []
    
    for word in words:
        if not word.strip():
            corrected_words.append(word)
            continue
        
        stats['total_words'] += 1
        
        if word in word_cache:
            stats['cached_hits'] += 1
            corrected_words.append(word_cache[word])
            continue
        
        if word not in spell_checker.unknown([word]):
            word_cache[word] = word
            corrected_words.append(word)
        else:
            corrected = list(spell_checker.correction(word))[0] if spell_checker.correction(word) else word
            
            if corrected != word:
                stats['corrected_words'] += 1
            
            word_cache[word] = corrected
            corrected_words.append(corrected)
    
    return ' '.join(corrected_words)

In [ ]:
stats['start_time'] = time.time()

tqdm.pandas(desc="pyspellchecker")
df['corrected'] = df['clean'].progress_apply(spell_check_py)

elapsed_time = time.time() - stats['start_time']
print_spell_check_stats(stats, elapsed_time)

pyspellchecker: 100%|██████████| 1000/1000 [35:20<00:00,  2.12s/it] 

Всего слов обработано: 88,529
Исправлено слов: 2,741
Процент исправленных слов: 3.10%
Попаданий в кэш: 81,487
Время выполнения: 2120.56 сек
Скорость: 42 слов/сек


Если проверять орфографию на сыром датасете без кэша, то выйдет ~36 минут при скорости 60 слов в секунду. Примерная [таблица сравнения](https://github.com/diffitask/spell-checkers-comparison)

#### Итого

Сохранил на всякий тут output  

pyspellchecker: 100%|██████████| 1000/1000 [35:20<00:00,  2.12s/it]  
Всего слов обработано: 88,529  
Исправлено слов: 2,741  
Процент исправленных слов: 3.10%  
Попаданий в кэш: 81,487  
Время выполнения: 2120.56 сек  
Скорость: 42 слов/сек  

Spylls: 100%|██████████| 1000/1000 [14:00<00:00,  1.19it/s]  
Всего слов обработано: 88,992  
Исправлено слов: 2,229  
Процент исправленных слов: 2.50%  
Попаданий в кэш: 81,947  
Время выполнения: 840.56 сек  
Скорость: 106 слов/сек  

В итоге Spylls сработал намного быстрее, хоть и исправил немного меньше слов

### Лемматизация

In [14]:
# Сделаем все слова в именительном падеже с помощью лемматизации для лучшей обработки
# Также обработаем отдельно слова с "не" как "не_слово" так как много слово по типу "не удовлетворительно", а nltk stopwords убирает "не"
# Применим PyMorphy3
from pymorphy3 import MorphAnalyzer

morph = MorphAnalyzer()

def lemmatize_text(text):
    raw_tokens = text.split()

    combined_tokens = []
    i = 0
    while i < len(raw_tokens):
        token = raw_tokens[i].strip()
        if token == 'не' and i + 1 < len(raw_tokens):
            next_token = raw_tokens[i + 1].strip()
            # Соединяем "не" со следующим словом
            combined_tokens.append('не_' + next_token)
            i += 2  # Пропускаем оба токена
        else:
            combined_tokens.append(token)
            i += 1

    tokens = []
    for token in combined_tokens:
        if token.startswith('не_'):
            word_to_check = token[2:]  # Убираем "не" для проверки
        else:
            word_to_check = token

        if token and word_to_check:
            # token = token.strip()
            # Получаем именительный падеж
            normal_form = morph.normal_forms(token)[0]
            tokens.append(normal_form)
    
    return ' '.join(tokens)

In [15]:
df['lemmatized_text'] = df['corrected'].apply(lemmatize_text)

In [16]:
# Сохраним, так как каждый раз по 15 минут проверять будет слишком долго
df.to_csv('text_cleaned_corrected.csv', index=False)

In [5]:
# Чтобы продолжить работу с уже очищенными данными
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df = pd.read_csv('data/processed/text_cleaned_corrected.csv')

## Выделение ключевых слов

### RAKE метод

In [3]:
from rake_nltk import Rake

def extract_keywords_rake(text, top_n=5):
    rake = Rake(language='russian')
    rake.extract_keywords_from_text(text)
    
    # Получаем топ-N фраз
    phrases = rake.get_ranked_phrases()[:top_n]
    
    # Возвращаем как строку
    return ' | '.join(phrases) if phrases else 'Нет ключевых слов'

In [ ]:
df_rake = df.copy()
df_rake['rake'] = df_rake['text'].apply(lambda x: extract_keywords_rake(x, top_n=5))
df_rake[['corrected', 'rake']].head(5)

,clean,rake
0,обратить взыскание денежные средства должника ковалев вячеслав геннадьевич г р находящиеся иных кредитных организациях юго западный юго западный кредит ооо курское гпб гпб центральный фк открытие независимо остатка денежных средств,денежные средства должника ковалев вячеслав геннадьевич 03 | филиал no 7701 банка втб | филиал no 7701 банка втб | филиал no 7701 банка втб | филиал no 7701 банка втб
1,обратить взыскание денежные средства должника арбузова михаила михайловича г р находящиеся иных кредитных организациях оренбургское оренбургское оренбургское оренбургское оренбургское оренбургское оренбургское акб абсолют,денежные средства должника арбузова михаила михайловича 03 | оренбургское отделение n8623 пао сбербанк | оренбургское отделение n8623 пао сбербанк | оренбургское отделение n8623 пао сбербанк | оренбургское отделение n8623 пао сбербанк
2,судебным приставом исполнителем обращено взыскание денежные средства должника кочетова сергея николаевича г р находящиеся иных кредитных организациях волгоградское южный райффайзенбанк райффайзенбанк акрополь южный райффайзенбанк южный райффайзенбанк волгоградское волгоградское волгоградское гпб гпб гпб гпб гпб,денежные средства должника кочетова сергея николаевича 17 | филиал no 7701 банка втб | филиал no 7701 банка втб | волгоградское отделение no8621 пао сбербанк | волгоградское отделение no8621 пао сбербанк
3,вынсено постановление запрет рег действий тс не указано не указано газ г г н ан номер шасси рамы номер кузова прицепа № двиг объём двигателя см куб мощность двигателя квт мощность двигателя л первичный птс уникальный номер регистрации ом наименование технологической операции изменение собственника владельца наименование регистрационного документа свидетельство регистрации транспортного средства приказ мвд россии № серия номер регистрационного документа выдачи регистрационного документа актуальности сведений,владельца ); наименование регистрационного документа свидетельство | наименование технологической операции изменение собственника | дата выдачи регистрационного документа 11 | приказ мвд россии № 346 | номер регистрационного документа 5959515244
4,рассмотрев доводы заявителя вирченко диана казбековна признает обстоятельства изложенные заявлении ходатайстве подлежащими удовлетворению рамках указанного исполнительного производства вынесены постановления обращении взыскания денежные средства должника почта ооо хкф юго западный краснодарский рф россельхозбанк краснодарский рф россельхозбанк центральный совкомбанк отп кредит ооо почта ооо хкф,"рамках указанного исполнительного производства вынесены постановления | вирченко диана казбековна признает обстоятельства | хкф банк "", бик 044525245 | хкф банк "", бик 044525245 | тинькофф банк "", бик 044525974"


Метод выдаёт самые часто встречающиеся, совместно встреченные слова по корпусу текста, но не учитывает важность некоторых слов в начале и конце, хоть и работает со стоп-словами. То есть не подходит для этого датасета, так как слишком много приказов, банков, имуществ в каждой строке

### YAKE метод

In [ ]:
import yake

yake_extractor = yake.KeywordExtractor (
    lan = "ru",     # язык
    n = 10,          # максимальное количество слов в фразе
    dedupLim = 0.8, # порог похожести слов
    top = 3         # количество ключевых слов
)

df['keywords'] = df['text'].apply(
    # lambda x: [kw for kw, score in yake_extractor.extract_keywords(x)]
    lambda x: '; '.join(kw for kw, score in yake_extractor.extract_keywords(x))
)

df[['corrected', 'keywords']].head()

,clean,answer,keywords
0,обратить взыскание денежные средства должника ковалев вячеслав геннадьевич г р находящиеся иных кредитных организациях юго западный юго западный кредит ооо курское гпб гпб центральный фк открытие независимо остатка денежных средств,Взыскание обращено,Обратить взыскание на денежные средства должника КОВАЛЕВ ВЯЧЕСЛАВ ГЕННАДЬЕВИЧ; взыскание на денежные средства должника КОВАЛЕВ ВЯЧЕСЛАВ ГЕННАДЬЕВИЧ; Обратить взыскание на денежные средства должника КОВАЛЕВ ВЯЧЕСЛАВ
1,обратить взыскание денежные средства должника арбузова михаила михайловича г р находящиеся иных кредитных организациях оренбургское оренбургское оренбургское оренбургское оренбургское оренбургское оренбургское акб абсолют,Взыскание обращено,Обратить взыскание на денежные средства должника Арбузова Михаила Михайловича; взыскание на денежные средства должника Арбузова Михаила Михайловича; Обратить взыскание на денежные средства должника Арбузова Михаила
2,судебным приставом исполнителем обращено взыскание денежные средства должника кочетова сергея николаевича г р находящиеся иных кредитных организациях волгоградское южный райффайзенбанк райффайзенбанк акрополь южный райффайзенбанк южный райффайзенбанк волгоградское волгоградское волгоградское гпб гпб гпб гпб гпб,Взыскание обращено,приставом-исполнителем обращено взыскание на денежные средства должника Кочетова Сергея Николаевича; Судебным приставом-исполнителем обращено взыскание на денежные средства должника Кочетова Сергея; обращено взыскание на денежные средства должника Кочетова Сергея Николаевича
3,вынсено постановление запрет рег действий тс не указано не указано газ г г н ан номер шасси рамы номер кузова прицепа № двиг объём двигателя см куб мощность двигателя квт мощность двигателя л первичный птс уникальный номер регистрации ом наименование технологической операции изменение собственника владельца наименование регистрационного документа свидетельство регистрации транспортного средства приказ мвд россии № серия номер регистрационного документа выдачи регистрационного документа актуальности сведений,Запрет действий,вынсено постановление на запрет рег.действий; вынсено постановление на запрет; постановление на запрет рег.действий
4,рассмотрев доводы заявителя вирченко диана казбековна признает обстоятельства изложенные заявлении ходатайстве подлежащими удовлетворению рамках указанного исполнительного производства вынесены постановления обращении взыскания денежные средства должника почта ооо хкф юго западный краснодарский рф россельхозбанк краснодарский рф россельхозбанк центральный совкомбанк отп кредит ооо почта ооо хкф,Постановление вынесено,производства вынесены постановления об обращении взыскания на денежные средства должника; рамках указанного исполнительного производства вынесены постановления об обращении взыскания; указанного исполнительного производства вынесены постановления об обращении взыскания на денежные


YAKE выводит хоть и отчасти нужные слова, но не всегда.  
YAKE состоит из 5 разных метрик, хоть метрика про позицию Position хорошо относится к этим данным из-за ответа в начале, но метрика про частоту Frequency наоборот выделяет множество слов по типу банк и повторяющиеся адреса.  

TF-IDF для выделения ключевых слов также не имеет смысла, так как работает от частоты вхождения слова в датасет

Хоть и не пробовал метод TextRank, в итоге нету смысла от выделения ключевых слов, так как уже обработанные данные подходят под rule-based метод, чтобы определить часть answer

## Классификация

### Rule метод

Так как в тексте могут быть явные указатели, при этом ответ почти всегда следует одному шаблону, запишем в answer уже явные ответы

In [2]:
import re

# Функции чтобы искать именно слова через "re", а не символы
# Хотя бы одно слово
def has_any_words(words, text):
    if not words:
        return False
    pattern = r'\b(' + '|'.join(re.escape(word) for word in words) + r')\b'
    return bool(re.search(pattern, text, flags=re.IGNORECASE))

# Хотя бы все слова
def has_all_words(words, text):
    for word in words:
        pattern = r'\b' + re.escape(word) + r'\b'
        if not re.search(pattern, text, flags=re.IGNORECASE):
            return False
    return True

def extract_status(text):
    if has_all_words(['подлежащее удовлетворение'], text):
        if has_any_words(['не_подлежащее', 'не_удовлетворение'], text):
            return 'Частично удовлетворено'
        return 'Удовлетворено'
    elif has_any_words(['не_подлежащее', 'не_удовлетворение'], text):
        if has_all_words(['подлежащее', 'удовлетворение'], text):
            return 'Частично удовлетворено'
        return 'Отказ'
    elif has_all_words(['ип', 'приостановлено'], text) or has_all_words(['ип', 'приостановить'], text):
        return 'ИП приостановлено'
    # elif has_any_words(['приостановить'], text):
    #     return 'Приостановлено'
    # elif has_all_words(['окончить', 'производство'], text):
    #     return 'Производство окончено'
    elif has_all_words(['обратить', 'взыскание'], text) or has_all_words(['обращение', 'взыскание',], text) and not 'не_иметься' in text:
        return 'Взыскание обращено'
    elif has_any_words(['действие', 'рег', 'регистрационный'], text) and 'запрет' in text:
        return 'Запрет действий'
    # elif has_all_words(['невозможно', 'установить'], text) or has_all_words(['не_представиться', 'лицо'], text) or has_all_words(['не_установить', 'должник'], text):
    #     return 'Отказ, невозможно установить местонахождение должника'
    elif has_all_words(['вынести', 'постановление'], text):
        return 'Постановление вынесено'
    elif has_all_words(['направить', 'запрос'], text):
        return 'Запрос направлен'
    elif has_all_words(['обращение', 'рассмотреть'], text) or has_all_words(['обращение', 'ответ'], text):
        return 'Обращение рассмотрено'
    else:
        return None

In [3]:
df['answer'] = df['lemmatized_text'].apply(extract_status)

In [4]:
print(df['answer'].value_counts())
print("Всего обработано строк:", df['answer'].value_counts().sum())

answer
Отказ                     195
Частично удовлетворено    116
Взыскание обращено        107
Обращение рассмотрено     100
Запрос направлен           81
Постановление вынесено     37
Удовлетворено              27
Запрет действий            15
Name: count, dtype: int64
Всего обработано строк: 678


Вывод на всякий:  
Отказ                                                    195  
Частично удовлетворено                                   116  
Взыскание обращено                                       106  
Обращение рассмотрено                                    100  
Запрос направлен                                          78  
Постановление вынесено                                    36  
Удовлетворено                                             27  
Запрет действий                                           14  
Отказ, невозможно установить местонахождение должника      8  
Производство окончено                                      7  
Приостановлено                                             1  
Всего обработано строк: 688  

Таким образом осталось ~треть данных из 1000 строк  
Остались только не входящие в правила строки и, возможно, неправильно обработанные

Также стоит убрать классы меньше чем с 10 примерами, так как они позже будут мешать обучению модели, а так при их удалении позже не могу исправить ошибку с индексами, что я и сделал, закомменировав код сверху в правилах.  
Если же переименовать такие классы на просто "другие", то модель может классифицировать многие тексты, как "другие", что не очень то является ответом на заявление

##### Через множество классов

In [5]:
import re

# Функции чтобы искать именно слова через "re", а не символы
# Хотя бы одно слово
def has_any_words(words, text):
    if not words:
        return False
    pattern = r'\b(' + '|'.join(re.escape(word) for word in words) + r')\b'
    return bool(re.search(pattern, text, flags=re.IGNORECASE))

# Хотя бы все слова
def has_all_words(words, text):
    for word in words:
        pattern = r'\b' + re.escape(word) + r'\b'
        if not re.search(pattern, text, flags=re.IGNORECASE):
            return False
    return True

def extract_status(text):
    statuses = []

    if has_all_words(['подлежащее удовлетворение'], text):
        statuses.append('Удовлетворено')
    if has_any_words(['не_подлежащее', 'не_удовлетворение'], text):
        if has_all_words(['подлежащее', 'удовлетворение'], text):
            statuses.append('Частично удовлетворено')
        statuses.append('Отказ')
    if has_all_words(['ип', 'приостановлено'], text):
        statuses.append('ИП приостановлено')
    if has_any_words(['приостановить'], text):
        statuses.append('Приостановлено')
    if has_all_words(['окончить', 'производство'], text):
        statuses.append('Производство окончено')
    if has_all_words(['обратить', 'взыскание'], text) or has_all_words(['обращение', 'взыскание'], text):
        statuses.append('Взыскание обращено')
    if has_any_words(['действие', 'рег', 'регистрационный'], text) and 'запрет' in text:
        statuses.append('Запрет действий')
    if has_all_words(['невозможно', 'установить'], text) or has_any_words(['не_установить', 'розыск'], text):
        statuses.append('Отказ, невозможно установить местонахождение должника')
    if has_all_words(['вынести', 'постановление'], text):
        statuses.append('Постановление вынесено')
    if has_all_words(['направить', 'запрос'], text):
        statuses.append('Запрос направлен')
    
    return statuses if statuses else None

In [6]:
df['answer'] = df['lemmatized_text'].apply(extract_status)

In [7]:
print(df['answer'].value_counts())
print("Всего обработано строк:", df['answer'].value_counts().sum())

answer
[Отказ]                                                                                                         130
[Запрос направлен]                                                                                               66
[Удовлетворено, Частично удовлетворено, Отказ]                                                                   66
[Отказ, невозможно установить местонахождение должника]                                                          41
[Взыскание обращено]                                                                                             35
                                                                                                               ... 
[Удовлетворено, Частично удовлетворено, Отказ, Взыскание обращено, Постановление вынесено, Запрос направлен]      1
[Отказ, Взыскание обращено, Отказ, невозможно установить местонахождение должника]                                1
[Отказ, Запрет действий, Постановление вынесено, Запрос направлен

##### Итог

На всякий вывел output  

[Отказ]                                                                               116  
[ИП приостановлено]                                                                    95  
[Запрос направлен]                                                                     54  
[Удовлетворено, Частично удовлетворено, Отказ]                                         45  
[Отказ, невозможно установить местонахождение должника]                                37  
                                                                                     ...   
[Отказ, Взыскание обращено, Отказ, невозможно установить местонахождение должника]      1  
[Отказ, Запрет действий, Постановление вынесено, Запрос направлен]                      1  
[Отказ, Взыскание обращено, Запрет действий, Постановление вынесено]                    1  
[ИП приостановлено, Взыскание обращено, Постановление вынесено, Запрос направлен]       1  
[Частично удовлетворено, Отказ]                                                         1  
Всего обработано строк: 714  

То есть вышло 87 классов через множество соединённых классов, что не вариант на этом датасете

Пока разметка классов идёт по первым встреченным словам.  
Проблема в том, что одна строка может иметь много классов, что переходит в многолейбловую классификацию, что не оставляет самые распространённые методы по типу SVM, Log Reg, K-means, а требует что-то похожее на MultiOutputClassifier.  
Также среди размеченных данных могут быть такие классы, которые встречаются только в числе других классов, а не одиночно, что значит не хватает размера датасета и модель просто не сможет правильно классифицировать.  

Можно сделать отдельные классы для совмещенных "Удовлетворительно + Взыскание обращено", но тогда классов будет намного больше, а датасет всё такой же маленький

In [5]:
df[['corrected', 'lemmatized_text', 'answer']].head()

,corrected,lemmatized_text,answer
0,обратить взыскание денежные средства должника Калевала Вячеслав Геннадьевич гс рт находящиеся иных кредитных организациях юго западный юго западный кредит ООО курское ГПУ ГПУ центральный к открытие независимо остатка денежных средств,обратить взыскание денежный средство должник калевала вячеслав геннадиевич гс рт находиться иной кредитный организация юго западный юго западный кредит ооо курский гпу гпу центральный к открытие независимо остаток денежный средство,Взыскание обращено
1,обратить взыскание денежные средства должника арбузов Михаила Михайловича гс рт находящиеся иных кредитных организациях оренбургское оренбургское оренбургское оренбургское оренбургское оренбургское оренбургское каб абсолют,обратить взыскание денежный средство должник арбуз михаил михаилович гс рт находиться иной кредитный организация оренбургский оренбургский оренбургский оренбургский оренбургский оренбургский оренбургский каб абсолют,Взыскание обращено
2,судебным приставом исполнителем обращено взыскание денежные средства должника кочетов Сергея Николаевича гс рт находящиеся иных кредитных организациях волгоградское южный райффайзенбанк райффайзенбанк акрополь южный райффайзенбанк южный райффайзенбанк волгоградское волгоградское волгоградское ГПУ ГПУ ГПУ ГПУ ГПУ,судебный пристав исполнитель обратить взыскание денежный средство должник кочет сергей николаевич гс рт находиться иной кредитный организация волгоградский южный райффайзенбанк райффайзенбанк акрополь южный райффайзенбанк южный райффайзенбанк волгоградский волгоградский волгоградский гпу гпу гпу гпу гпу,Взыскание обращено
3,вынесено постановление запрет ре действий ст не указано не указано газ гс гс ню ан номер шасси рамы номер кузова прицепа № сдвиг объём двигателя см куб мощность двигателя кВт мощность двигателя ля первичный пс уникальный номер регистрации ом наименование технологической операции изменение собственника владельца наименование регистрационного документа свидетельство регистрации транспортного средства приказ МВД России № серия номер регистрационного документа выдачи регистрационного документа актуальности сведений,вынести постановление запрет ре действие ст не_указать не_указать газ гс гс ню ан номер шасси рама номер кузов прицеп № сдвиг объём двигатель смотреть куб мощность двигатель квт мощность двигатель ля первичный пс уникальный номер регистрация ом наименование технологический операция изменение собственник владелец наименование регистрационный документ свидетельство регистрация транспортный средство приказ мвд россия № серия номер регистрационный документ выдача регистрационный документ актуальность сведение,Запрет действий
4,рассмотрев доводы заявителя Кравченко Диана азбуковника признает обстоятельства изложенные заявлении ходатайстве подлежащими удовлетворению рамках указанного исполнительного производства вынесены постановления обращении взыскания денежные средства должника почта ООО мкФ юго западный краснодарский РФ сельхозартель краснодарский РФ сельхозартель центральный совком банк топ кредит ООО почта ООО мкФ,рассмотреть довод заявитель кравченко диана азбуковник признать обстоятельство изложить заявление ходатайство подлежащее удовлетворение рамка указанный исполнительный производство вынести постановление обращение взыскание денежный средство должник почта ооо мкф юго западный краснодарский рф сельхозартель краснодарский рф сельхозартель центральный совок банк топ кредит ооо почта ооо мкф,Удовлетворено


### Naive Bayes, Logistic Regression, SVM

Обучим алгоритмы на тех данных, которые уже имеют верные метки

In [6]:
df_labeled = df[df['answer'].notna()].copy()
# original_indices_labeled = df_labeled.index.copy() # Для последующего объеденения с предсказаниями
df_labeled = df_labeled.reset_index(drop=True)

df_labeled['answer'].value_counts()

answer
Отказ                     195
Частично удовлетворено    116
Взыскание обращено        107
Обращение рассмотрено     100
Запрос направлен           81
Постановление вынесено     37
Удовлетворено              27
Запрет действий            15
Name: count, dtype: int64

In [7]:
# Обучение моделей не выдаёт ошибок, если оставить классы с больше 40 примерами и если использовать class_weight='balanced',
# но тогда отсекаем 5 из 11 классов, можно оставить максимум 8, так как 14 примеров могут показаться достаточными
# Может стоит позже попробовать аугментировать данные с SMOTE, хотя классы слишком редкие

# Оставить только классы с >= 10 примерами, так как слишком мало классов для любой модели
# min_samples = 10
# class_counts = df_labeled['answer'].value_counts()
# valid_classes = class_counts[class_counts >= min_samples].index

# df_labeled = df_labeled[df_labeled['answer'].isin(valid_classes)]

# df_labeled['answer'].value_counts()

In [8]:
from sklearn.model_selection import train_test_split

X = df_labeled['lemmatized_text']
y = df_labeled['answer']

# Может стоит попробовать Stratified K-Fold, раз классы не сбалансированы
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=1,
    stratify=y
)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

classifiers = [
    ('Naive Bayes', MultinomialNB()),
    ('Logistic Regression', LogisticRegression(max_iter=1000, 
                                               random_state=1, 
                                               class_weight='balanced')), # балансируем, так как слишком редкие классы
    ('SVM', LinearSVC(max_iter=2000, 
                      random_state=1, 
                      class_weight='balanced')) # балансируем, так как слишком редкие классы
]

for name, classifier in classifiers:
    print(f"\n{'='*100}")
    print(f"Модель: {name}")
    print(f"{'='*100}")
    
    # Создаем pipeline для каждой модели
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=1000,
                                ngram_range=(1, 2))),
        ('classifier', classifier)
    ])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Точность: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))


Модель: Naive Bayes
Точность: 0.7451
                        precision    recall  f1-score   support

    Взыскание обращено       0.59      0.69      0.64        32
       Запрет действий       0.00      0.00      0.00         5
      Запрос направлен       0.65      0.46      0.54        24
 Обращение рассмотрено       0.66      0.90      0.76        30
                 Отказ       0.84      0.97      0.90        59
Постановление вынесено       0.00      0.00      0.00        11
         Удовлетворено       0.00      0.00      0.00         8
Частично удовлетворено       0.85      1.00      0.92        35

              accuracy                           0.75       204
             macro avg       0.45      0.50      0.47       204
          weighted avg       0.66      0.75      0.69       204


Модель: Logistic Regression


c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Programming\text_classification\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

Точность: 0.8578
                        precision    recall  f1-score   support

    Взыскание обращено       0.81      0.81      0.81        32
       Запрет действий       0.25      0.20      0.22         5
      Запрос направлен       0.64      0.75      0.69        24
 Обращение рассмотрено       0.87      0.90      0.89        30
                 Отказ       0.98      0.97      0.97        59
Постановление вынесено       0.50      0.27      0.35        11
         Удовлетворено       0.80      1.00      0.89         8
Частично удовлетворено       1.00      1.00      1.00        35

              accuracy                           0.86       204
             macro avg       0.73      0.74      0.73       204
          weighted avg       0.85      0.86      0.85       204


Модель: SVM
Точность: 0.8578
                        precision    recall  f1-score   support

    Взыскание обращено       0.82      0.88      0.85        32
       Запрет действий       0.33      0.20      0.25

UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior. _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])

Ошибка возникает, так как модель Naive Bayes не предсказал ни одного примера редких классов, SVM и Log Reg же попытались, хоть и не очень удачно по precision, recall, f1

SVM и Log Reg имеют лучшие показатели точности accuracy, но в этом случае это не показывает качество модели из-за несбалансированных классов.
NB имеет худшие результаты, он не определяет самые редкие классы, впрочем SVM и Log Reg методы так же имеют плохие recall и f1 score на тех же классах, хоть и показывают хорошие macro avg и weighted avg

### SMOTE

Попробуем увеличить редкие классы

In [10]:
# Посмотрим сколько будет пример до и после
from imblearn.over_sampling import SMOTE

vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)

smote = SMOTE(random_state=1, sampling_strategy='not majority')
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_tfidf, y_train)

print("Размер классов train до:", y_train.value_counts())
print("\nРазмер классов train после:", y_train_balanced.value_counts())

print("\nРазмер train до:", len(y_train))
print("Размер train после:", len(y_train_balanced))
print("Добавлено всего:", len(y_train_balanced) - len(y_train))

Размер классов train до: answer
Отказ                     136
Частично удовлетворено     81
Взыскание обращено         75
Обращение рассмотрено      70
Запрос направлен           57
Постановление вынесено     26
Удовлетворено              19
Запрет действий            10
Name: count, dtype: int64

Размер классов train после: answer
Запрет действий           136
Частично удовлетворено    136
Обращение рассмотрено     136
Отказ                     136
Запрос направлен          136
Постановление вынесено    136
Взыскание обращено        136
Удовлетворено             136
Name: count, dtype: int64

Размер train до: 474
Размер train после: 1088
Добавлено всего: 614


In [11]:
from imblearn.pipeline import Pipeline as ImbPipeline
# import pickle

trained_pipelines = {}

classifiers = [
    ('Naive Bayes', MultinomialNB()),
    ('Logistic Regression', LogisticRegression(max_iter=1000,
                                               random_state=1,
                                               class_weight='balanced')), # балансируем, так как слишком редкие классы
    ('SVM', LinearSVC(max_iter=2000, 
                      random_state=1, 
                      class_weight='balanced')) # балансируем, так как слишком редкие классы
]

for name, classifier in classifiers:
    print(f"\n{'='*100}")
    print(f"Модель: {name}")
    print(f"{'='*100}")
    
    # Создаем pipeline для каждой модели
    pipeline = ImbPipeline([
        ('tfidf', TfidfVectorizer(max_features=1000,
                                ngram_range=(1, 2))),
        ('smote', SMOTE(
            random_state=1,
            sampling_strategy='not majority'
        )),
        ('classifier', classifier)
    ])
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Точность: {accuracy:.4f}")
    print(classification_report(y_test, y_pred))

    trained_pipelines[name] = pipeline

    # with open(f'pipeline_{name}.pkl', 'wb') as f:
    #     pickle.dump(pipeline, f)


Модель: Naive Bayes
Точность: 0.8137
                        precision    recall  f1-score   support

    Взыскание обращено       0.78      0.66      0.71        32
       Запрет действий       0.33      0.40      0.36         5
      Запрос направлен       0.61      0.71      0.65        24
 Обращение рассмотрено       0.75      0.90      0.82        30
                 Отказ       0.98      0.92      0.95        59
Постановление вынесено       0.40      0.18      0.25        11
         Удовлетворено       0.73      1.00      0.84         8
Частично удовлетворено       0.97      1.00      0.99        35

              accuracy                           0.81       204
             macro avg       0.69      0.72      0.70       204
          weighted avg       0.81      0.81      0.81       204


Модель: Logistic Regression
Точность: 0.8627
                        precision    recall  f1-score   support

    Взыскание обращено       0.82      0.84      0.83        32
       Запрет де

SMOTE помог только Naive Bayes методу предсказать ранее не предсказанные классы, и повысить все метрики.  
Метрики же на других методах не сильно поменялись или вообще стали чуть хуже

Так как метрики относительно одинаковы между Log Reg и SVM, можно выбрать одну из них. Однако Naive Bayes выдаёт иные метрики, некоторые лучше, в основном хуже. Можно попробовать ансамбль моделей

### Классификация неразмеченного текста

In [ ]:
df_unlabeled = df[df['answer'].isna()].copy()
# original_indices_unlabeled = df_labeled.index.copy()
df_unlabeled = df_unlabeled.reset_index(drop=True) # Нужно обнулить индексы, так как предсказывает только 100 из 322 почему-то
X_unlabeled = df_unlabeled['lemmatized_text']

predictions_all = {}
for name, pipeline in trained_pipelines.items():
    predictions_all[name] = pipeline.predict(X_unlabeled)

predictions_df = pd.DataFrame(predictions_all)
# Перевернуть столбцы, чтобы из-за mode, которая при всех разных вариантов выбирает первый столбце в списке, выбиралась лучшая модель - SVM
predictions_df = predictions_df[['SVM', 'Logistic Regression', 'Naive Bayes']]

print("Датафрейм с предсказаниями от всех моделей:")
print(predictions_df.head())

# Выбрать наиболее частый ответ среди всех строк
df_unlabeled['answer'] = predictions_df.mode(axis=1)[0]

print("\nКоличество предсказанных классов:", df_unlabeled['answer'].value_counts())

Датафрейм с предсказаниями от всех моделей:
                      SVM     Logistic Regression             Naive Bayes
0  Постановление вынесено  Постановление вынесено  Постановление вынесено
1        Запрос направлен                   Отказ        Запрос направлен
2   Обращение рассмотрено   Обращение рассмотрено           Удовлетворено
3   Обращение рассмотрено                   Отказ                   Отказ
4      Взыскание обращено      Взыскание обращено           Удовлетворено

Количество предсказанных классов: answer
Обращение рассмотрено     166
Постановление вынесено     53
Запрос направлен           49
Взыскание обращено         40
Отказ                       8
Запрет действий             6
Name: count, dtype: int64


In [141]:
df_unlabeled[['answer', 'lemmatized_text']].head(3)

,answer,lemmatized_text
0,Постановление вынесено,взыскивать денежный средство перечислять следующий реквизит получатель ооо мкклый свериться расчётный получатель инна ооо мкклый свериться – кпд получатель ооо мкклый свериться инна кпд корсет занести исполнительный производство такой образ постановление действие судебный пристав исполнитель оп хунхузский распутинский тамильский район являться правомерный
1,Запрос направлен,дать тю внести бдеть ас мобильный розыск дорожный пристать осуществляться еженедельный рейд совместно орган мрамор гибрид выявление залоговый тю направляться требование должник установление местонахождение залоговый тю дополнительно осуществляться выход предполагать место расположение тю также ежемесячно проверяться система отслеживание тю безопасный город
2,Обращение рассмотрено,согласно электронный ответ мрамор гибрид автотранспортный средство также сельскохозяйственный самоходный техника должник не_зарегистрировать согласно ответ возрождение «» «мдм » мниться » лето уральский смотреть каб полбанки скоба раструсить ооо «» каб экспресс волга райффайзенбанк россия – гс москва москва газпром гс ставрополь перегрев ставропольпромстройбанк должник не_являться клиент указанный кредитный учреждение открытый счёт


#### Итоговый датасет

In [13]:
df_labeled.index = df[df['answer'].notna()].index
df_unlabeled.index = df[df['answer'].isna()].index

In [18]:
df_result = pd.concat([df_labeled, df_unlabeled]).sort_index()

print(df.index)
print(df_result.index)

RangeIndex(start=0, stop=1000, step=1)
Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
       ...
       990, 991, 992, 993, 994, 995, 996, 997, 998, 999],
      dtype='int64', length=1000)


Индексы ~совпадают

In [ ]:
# Сохраню в xlsx, так как много строк имеют \n и \r символов, что ломает разделение строк в csv
df_result[['Id', 'text', 'answer']].to_excel('data/processed/result.xlsx', index=False)
df_result.to_excel('data/processed/result_detailed.xlsx', index=False)

## Label Propagation

In [40]:
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.preprocessing import LabelEncoder